# Generate PA plan and breaks between sedentary bouts for the coming week

!!! TODO:
1. reference for weekly patterns (today) (done)
2. consider possible scenarios for unrecognizable patterns (tomorrow)
3. cutoff threshold for good predictability (sunday) (done)
4. test accuracy
5. for prediction, filter out missing data

Prompt the user to sync data and run this script every Monday morning at 9am.

1. get date of today
2. pull data from the last 7 weeks or all available data less than 7 weeks
3. get user preference for total MET and mode (MVPA/VPA)
4. generate PA plan
5. get calendar data for the week
6. populate "busy" intervals
7. predict time to send prompts to break up sedentary bouts

In [1]:
import pandas as pd
import json
import glob
import math
import datetime as dt
from pandas.tseries.offsets import *
# import configparser
# import os
# from datetime import datetime
# from xlrd.xldate import xldate_as_tuple


##############
# Script 1 > #
##############
# setting up basic variables
# run date
today = '2016-09-26' # for simulation only, for production change to dt.datetime.today().strftime("%Y-%m-%d")
# timestamps in a day
timestamp_list = pd.date_range(today, periods=96, freq='15min').strftime('%H:%M:%S')
# generate dates for pulling train data ### TODO: find reference for how many weeks of data to pull for train 
weeks_to_pull = 4
train_start_date = pd.to_datetime(today) - DateOffset(weeks=weeks_to_pull)
train_date_range = pd.date_range(train_start_date, periods=(7*weeks_to_pull), freq='D').strftime('%Y-%m-%d')
### TODO: send train_date_list to App and retrieve training data
##############
# < Script 1 #
##############

train_date_range

array(['2016-08-29', '2016-08-30', '2016-08-31', '2016-09-01',
       '2016-09-02', '2016-09-03', '2016-09-04', '2016-09-05',
       '2016-09-06', '2016-09-07', '2016-09-08', '2016-09-09',
       '2016-09-10', '2016-09-11', '2016-09-12', '2016-09-13',
       '2016-09-14', '2016-09-15', '2016-09-16', '2016-09-17',
       '2016-09-18', '2016-09-19', '2016-09-20', '2016-09-21',
       '2016-09-22', '2016-09-23', '2016-09-24', '2016-09-25'], 
      dtype='<U10')

##############
# Script 2 > #
##############
# Input:
# 1. Training data
# 2. date range of training data: today, train_date_range
# 3. User profile (ID, age, exercise choice, ...)
# Output:
# 1. userData
# 2. dates of missing file (check and redownload)
# setting up basic variables
# run date
today = '2016-09-26' # for simulation only, for production change to dt.datetime.today().strftime("%Y-%m-%d")
# timestamps in a day
timestamp_list = pd.date_range(today, periods=96, freq='15min').strftime('%H:%M:%S')
# file path and file names
heart_files = glob.glob("heart*.json")
# generate dates for pulling train data ### TODO: find reference for how many weeks of data to pull for train 
weeks_to_pull = 4
train_start_date = pd.to_datetime(today) - DateOffset(weeks=weeks_to_pull)
train_date_range = pd.date_range(train_start_date, periods=(7*weeks_to_pull), freq='D').strftime('%Y-%m-%d')

# prepare training data
def prepare_data(date_list=train_date_range, user_id='a001'):
    # returns userData['user_id', 'week', 'weekday', 'date', 'time', 'time_count', 'heart', 'step']
    list_ = []
    file_missing = []
    for d in date_list:
        heart_data_exist = True
        step_data_exist = True
        # prepare names of files to load
        heart_fname = 'heart'+ d + '.json'
        step_fname = 'step'+ d + '.json'
        try:
            with open(heart_fname) as heart_data_file:    
                heart_data = json.load(heart_data_file)
            heart_data = pd.DataFrame.from_dict(heart_data['activities-heart-intraday']['dataset'])
            heart_data = heart_data.set_index('time')
        except:
            # TODO: add ErrorType
            # file could not be found
            heart_data_exist = False
            print('%s could not be found' % heart_fname)
        try:
            with open(step_fname) as step_data_file:    
                step_data = json.load(step_data_file)
            step_data = pd.DataFrame.from_dict(step_data['activities-steps-intraday']['dataset'])
            step_data = step_data.set_index('time')
        except:
            # TODO: add ErrorType
            # file could not be found
            step_data_exist = False
            print('%s could not be found' % step_fname)
            
        # fill in missing data with 0 TODO: or NaN?
        if (heart_data_exist & step_data_exist):
            complete_data = pd.DataFrame({'time': timestamp_list,
                                          'time_count': range(len(timestamp_list)),
                                          'heart': 0*len(timestamp_list), 
                                          'step': 0*len(timestamp_list), 
                                          'date': d}, index = timestamp_list)
            for i in complete_data.index:
                try:
                    complete_data.loc[i, 'heart'] = heart_data.loc[i, 'value']
                    complete_data.loc[i, 'step'] = step_data.loc[i, 'value']
                except:
                    next
            list_.append(complete_data)
        else:
            file_missing.append(d)
    userData = pd.concat(list_)
    
    # final preparation for userData
    userData['user_id'] = user_id
    weekday = []
    week = []
    for i in userData.date:
        weekday.append(pd.to_datetime(i).dayofweek)
        week.append(pd.to_datetime(i).week)
    userData['weekday'] = weekday
    userData['week'] = week
    return userData, file_missing


userData, file_missing = prepare_data()
print(userData.head(100)) ## this is the orginal copy. DO NOT modify.
print(file_missing)

# prepare test data
test_date_range = pd.date_range(today, periods=(7*2), freq='D').strftime('%Y-%m-%d')
testData, test_file_missing = prepare_data(date_list=test_date_range)
print(testData.head()) ## this is the orginal copy. DO NOT modify.
print(test_file_missing)
##############
# < Script 2 #
##############

## evaluate activity level MVPA, VPA, MET

#### based on heart rate (HPB)



In [4]:
##############
# Script 3 > #
##############
# Input:
# 1. userData
# 2. userProfile
# Output:
# pa plan for the next 7 days
class User:
    def __init__(self, age, choice, goal, completed):
        self.age = age
        self.choice = choice
        self.max_hr = 220 - self.age
        self.mvpa_low = 0.64 * self.max_hr
        self.mvpa_high = 0.77 * self.max_hr
        self.vpa_low = 0.77 * self.max_hr
        self.vpa_high = 0.89 * self.max_hr
        self.goal = goal
        self.completed = completed
        self.current_goal = self.goal - self.completed # current goal should be based on goal from previous week minus met completed this week
    
import copy

# User Settings

In [5]:
# specify user settings
user = User(21, 'MVPA', 600, 0)
user.mvpa_low
user.current_goal

600

In [6]:
# prepare a copy of PA data
userPA = copy.deepcopy(userData)  # date  heart  step      time  time_count user_id  weekday  week

# mark out MVPA intervals
def find_mvpa(user_data=userPA, user_settings=user):
    mvpa_select = (user_data.heart >= user_settings.mvpa_low) & (user_data.heart < user_settings.mvpa_high) & (user_data.step > 0)
    user_data['mvpa_select'] = mvpa_select
    user_data['mvpa'] = 0
    user_data.loc[user_data.mvpa_select, 'mvpa'] = 1
    user_data.drop('mvpa_select', axis=1, inplace=True)
    return user_data

# mark out VPA intervals
def find_vpa(user_data=userPA, user_settings=user):
    vpa_select = (user_data.heart >= user_settings.vpa_low) & (user_data.heart < user_settings.vpa_high) & (user_data.step > 0)
    user_data['vpa_select'] = vpa_select
    user_data['vpa'] = 0
    user_data.loc[user_data.vpa_select, 'vpa'] = 1
    user_data.drop('vpa_select', axis=1, inplace=True)
    return user_data

# calculate MET
def calculate_met(user_data=userPA):
    user_data['met'] = user_data['mvpa'] * 4 * 15 + user_data['vpa'] * 8 * 15
    return user_data

userPA = find_mvpa(userPA)
userPA = find_vpa(userPA)
userPA = calculate_met(userPA)
# drop rows where data are missing
userPA = userPA[(userPA.heart > 0) & (userPA.step > 0)]

In [10]:
userPA[userPA['mvpa'] > 0]
# get the last 7 weeks' data. If less than 7 weeks, get all data available. 
# (7 days required for sedentary, but that is for one week)
# DF structure: userID, date, heartrate, step, time, time_index, weekday, week, MVPA, VPA, MET, sedentary, busy

,date,heart,step,time,time_count,user_id,weekday,week,mvpa,vpa,met
14:45:00,2016-09-02,131,708,14:45:00,59,a001,4,35,1,0,60
18:45:00,2016-09-05,140,1237,18:45:00,75,a001,0,36,1,0,60
19:30:00,2016-09-10,139,2006,19:30:00,78,a001,5,36,1,0,60
19:45:00,2016-09-10,145,2099,19:45:00,79,a001,5,36,1,0,60
20:00:00,2016-09-10,133,1317,20:00:00,80,a001,5,36,1,0,60
21:45:00,2016-09-19,129,2,21:45:00,87,a001,0,38,1,0,60
00:45:00,2016-09-24,128,17,00:45:00,3,a001,5,38,1,0,60


### 1. Determine if MET distribution is correlated to day of week
### 2. if yes: use A2; if no: use random assign


In [42]:
# experiment 1
x = pd.DataFrame({'weekday': [0,1,2,3,4,5] * 4, 'met': [0, 0, 0, 0, 0, 60] * 4})
x.corr(method='spearman')
spearmanr(x['weekday'], x['met'])[1] # p value <= 0.05 statistically significant

0.00051848748583231729

In [77]:
# experiment 2
# when input values are random numbers, accumulated met is not correlated with weekdays
import numpy as np
x = pd.DataFrame({'week': np.repeat([0,1,2,3],6),'weekday': [0,1,2,3,4,5] * 4, 'met': np.random.randint(0,100,size=24)})
x['accumulated'] = x['met']
i = 0
while i < len(x)-1:
    if x.loc[i, 'week'] == x.loc[i+1, 'week']:
        x.loc[i+1, 'accumulated'] += x.loc[i, 'accumulated']
    i += 1
print(x)
x.corr(method='spearman')
print('corr is %f' % spearmanr(x['weekday'], x['met'])[0])
print('p-value is %f' % spearmanr(x['weekday'], x['met'])[1]) # p value <= 0.05 statistically significant


    met  week  weekday  accumulated
0    93     0        0           93
1    61     0        1          154
2    80     0        2          234
3    13     0        3          247
4    89     0        4          336
5    21     0        5          357
6    94     1        0           94
7    78     1        1          172
8    13     1        2          185
9    13     1        3          198
10   42     1        4          240
11   83     1        5          323
12   38     2        0           38
13   68     2        1          106
14   71     2        2          177
15   69     2        3          246
16    8     2        4          254
17   22     2        5          276
18   20     3        0           20
19   76     3        1           96
20    1     3        2           97
21   59     3        3          156
22   32     3        4          188
23    4     3        5          192
corr is -0.328067
p-value is 0.117566


In [11]:
from scipy.stats.stats import spearmanr, pearsonr
def get_pa_pattern(user_data=userPA):
    # get daily total MET
    daily_met_grouped = user_data.groupby(['week', 'weekday'])
    daily_met = pd.DataFrame(daily_met_grouped['met'].sum()).reset_index()
    # get accumulated met of each day, starting from Monday
    daily_met['accumulated'] = daily_met['met']
    i = 0
    while i < len(daily_met)-1:
        if daily_met.loc[i, 'week'] == daily_met.loc[i+1, 'week']:
            daily_met.loc[i+1, 'accumulated'] += daily_met.loc[i, 'accumulated']
        i += 1
    print(daily_met)
    # check correlation between day of week and MET
    check_corr_weekday_met = daily_met[['weekday', 'accumulated']]
    corr_weekday_met = spearmanr(daily_met['weekday'], daily_met['accumulated'])[0]
    p_weekday_met = spearmanr(daily_met['weekday'], daily_met['accumulated'])[1]
    print('p value is %f' % p_weekday_met)
    print('corr is %f' % corr_weekday_met)
    if (p_weekday_met <= 0.01) & (abs(corr_weekday_met) > 0):
        strategy = 'A2' # correlated
    else:
        strategy = 'A1' # not correlated
    return strategy
    
get_pa_pattern()

    week  weekday  met  accumulated
0     35        0    0            0
1     35        1    0            0
2     35        2    0            0
3     35        3    0            0
4     35        4   60           60
5     35        5    0           60
6     35        6    0           60
7     36        0   60           60
8     36        1    0           60
9     36        2    0           60
10    36        3    0           60
11    36        4    0           60
12    36        5  180          240
13    36        6    0          240
14    37        0    0            0
15    37        1    0            0
16    37        2    0            0
17    37        3    0            0
18    37        4    0            0
19    37        5    0            0
20    37        6    0            0
21    38        0   60           60
22    38        1    0           60
23    38        2    0           60
24    38        3    0           60
25    38        4    0           60
26    38        5   60      

'A1'

# evaluate correlation between PA of yesterday and PA of today

In [18]:
# use userPA

daily_met_grouped = userPA.groupby(['week', 'weekday'])
daily_met = pd.DataFrame(daily_met_grouped['met'].sum()).reset_index()
daily_met['met'].unique()
# probability of if yesterday no PA today also no PA
# assume no PA the day before the first data point. put the check of the first data point at the end.
count_0_0 = 0
count_0_1 = 0
count_1_0 = 0
count_1_1 = 0
n = 0
while n < len(daily_met) - 1:
    m = n + 1
    if (daily_met['met'][n] == 0)&(daily_met['met'][m] == 0):
        count_0_0 += 1
    elif (daily_met['met'][n] == 0)&(daily_met['met'][m] != 0):
        count_0_1 += 1
    elif (daily_met['met'][n] != 0)&(daily_met['met'][m] == 0):
        count_1_0 += 1
    elif (daily_met['met'][n] != 0)&(daily_met['met'][m] != 0):
        count_1_1 += 1
    n += 1

p_0_0 = count_0_0/len(daily_met)
p_0_1 = count_0_1/len(daily_met)
p_1_0 = count_1_0/len(daily_met)
p_1_1 = count_1_1/len(daily_met)
print(p_0_0)
print(p_0_1)
print(p_1_0)
print(p_1_1)
p_0 = len(daily_met[daily_met['met'] == 0])/len(daily_met)
p_1 = len(daily_met[daily_met['met'] > 0])/len(daily_met)
print(p_0)
print(p_1)
p_given1_0 =  p_1_0/p_1
print(p_given1_0)

0.6071428571428571
0.17857142857142858
0.17857142857142858
0.0
0.8214285714285714
0.17857142857142858
1.0


In [15]:
len(daily_met[daily_met['met'] > 0])

5

### A1 (the distribution does not show strong weekly pattern of PA)

In [9]:
def get_a1_plan(goal=user.current_goal, choice=user.choice):
    # model of randomly assign PA of choice to days left in this week to fulfill MET goal left in this week
    # generate days left in this week
    dayofweek_today = pd.to_datetime(dt.datetime.today()).dayofweek
    # no more PA should be arranged after certain time, let's say 10pm for now
    if dt.datetime.today().time().hour >= 22:
        days_left = pd.Series(list(range(dayofweek_today + 1,7)))
    else:
        days_left = pd.Series(list(range(dayofweek_today,7)))
    # select x days out of daysPArequired
    if choice == 'MVPA':
        numofbouts = math.ceil(goal/(4 * 15))
    elif choice == 'VPA':
        numofbouts = math.ceil(goal/(8 * 15))
    base_bouts = math.floor(numofbouts/len(days_left))
    extra_bouts = numofbouts % len(days_left)
    plan = pd.DataFrame({'weekday': days_left,
                         'date': pd.date_range(dt.datetime.today(), periods=7, freq='D').strftime('%Y-%m-%d'),
                         'choice': choice, 'bouts': base_bouts})
    days_extra_bouts = days_left.sample(extra_bouts)
    for i in days_extra_bouts:
        plan.loc[plan['weekday'] == i, 'bouts'] += 1
    return plan

get_a1_plan()

ValueError: array length 7 does not match index length 6

# A2: user's PA show an pattern, find days less active

In [134]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn import linear_model

def get_a2_plan(user_data=userPA, goal=user.current_goal, choice=user.choice):
    # model of randomly assign PA of choice to less active days identified left in this week to fulfill MET goal left in this week
    # get daily total MET
    daily_met_grouped = user_data.groupby(['week', 'weekday'])
    daily_met = pd.DataFrame(daily_met_grouped['met'].sum()).reset_index()
    # logitic regression
    # get accumulated met of each day, starting from Monday
    daily_met['accumulated'] = daily_met['met']
    i = 0
    while i < len(daily_met)-1:
        if daily_met.loc[i, 'week'] == daily_met.loc[i+1, 'week']:
            daily_met.loc[i+1, 'accumulated'] += daily_met.loc[i, 'accumulated']
        i += 1
    print(daily_met)
    '''
    method 1: decision tree
    # predict this week's PA trend
    a2_tree=DecisionTreeClassifier()
    a2_tree = a2_tree.fit(np.array(daily_met[['weekday']]), daily_met.loc[:, 'met'])
    predicted = pd.DataFrame({'weekday': list(range(7)), 'predicted_met': 0})
    print(a2_tree.predict(predicted[['weekday']]))
    print(a2_tree.predict_proba(predicted[['weekday']]))
    
    method 2: linear regression
    # Create linear regression object
    regr = linear_model.LinearRegression()

    # Train the model using the training sets
    regr.fit(daily_met.loc[:, 'weekday'].reshape(-1,1), daily_met.loc[:, 'met'].reshape(-1,1))

    # The coefficients
    print('Coefficients: \n', regr.coef_)
    print('Variance score: %.2f' % regr.score(daily_met.loc[:, 'weekday'].reshape(-1,1), daily_met.loc[:, 'met'].reshape(-1,1)))

    # Plot outputs
    plt.scatter(daily_met.loc[:, 'weekday'].reshape(-1,1), daily_met.loc[:, 'met'].reshape(-1,1),  color='black')
    plt.plot(daily_met.loc[:, 'weekday'].reshape(-1,1), regr.predict(daily_met.loc[:, 'weekday'].reshape(-1,1)), color='blue',
             linewidth=3)

    plt.xticks(())
    plt.yticks(())

    plt.show()
    
    method 3: logistic regression by classifying met to high (1) and low (0)
    
    method 4: conditional probability
    
    '''
    
    # method 4: conditional prob of less active days
    predicted_ = pd.DataFrame({'weekday': [0,1,2,3,4,5,6]})
    require_pa = []
    for i in list(range(7)):
        prob_day = len(daily_met[daily_met['weekday'] == i])/len(daily_met)
        prob_notactiveday = len(daily_met[(daily_met['weekday'] == i) & (daily_met['met'] == 0)])/len(daily_met)
        prob_notactiveontheday = prob_notactiveday/prob_day
        if prob_notactiveontheday > 0.5:  # likely to be inactive
            require_pa.append(1)
        else:
            require_pa.append(0)
    predicted_['require_pa'] = require_pa
    print(predicted_)
get_a2_plan()


    week  weekday  met  accumulated
0     35        0    0            0
1     35        1    0            0
2     35        2    0            0
3     35        3    0            0
4     35        4   60           60
5     35        5    0           60
6     35        6    0           60
7     36        0   60           60
8     36        1    0           60
9     36        2    0           60
10    36        3    0           60
11    36        4    0           60
12    36        5  180          240
13    36        6    0          240
14    37        0    0            0
15    37        1    0            0
16    37        2    0            0
17    37        3    0            0
18    37        4    0            0
19    37        5    0            0
20    37        6    0            0
21    38        0   60           60
22    38        1    0           60
23    38        2    0           60
24    38        3    0           60
25    38        4    0           60
26    38        5   60      

In [197]:
testPA = copy.deepcopy(testData)
testPA = find_mvpa(testPA)
testPA = find_vpa(testPA)
testPA = calculate_met(testPA)
test_daily_met_grouped = testPA.groupby(['week', 'weekday'])
test_daily_met = pd.DataFrame(test_daily_met_grouped['met'].sum()).reset_index()
print(test_daily_met)

    week  weekday  met
0     39        0    0
1     39        1    0
2     39        2    0
3     39        3    0
4     39        4    0
5     39        5    0
6     39        6    0
7     40        0    0
8     40        1    0
9     40        2  180
10    40        3    0
11    40        4    0
12    40        5    0
13    40        6    0


In [198]:
    
    
    
    # generate days left in this week
    dayofweek_today = pd.to_datetime(dt.datetime.today()).dayofweek
    # no more PA should be arranged after certain time, let's say 10pm for now
    if dt.datetime.today().time().hour >= 22:
        days_left = pd.Series(list(range(dayofweek_today + 1,7)))
    else:
        days_left = pd.Series(list(range(dayofweek_today,7)))
    # select x days out of daysPArequired
    if choice == 'MVPA':
        numofbouts = math.ceil(goal/(4 * 15))
    elif choice == 'VPA':
        numofbouts = math.ceil(goal/(8 * 15))
    base_bouts = math.floor(numofbouts/len(days_left))
    extra_bouts = numofbouts % len(days_left)
    plan = pd.DataFrame({'weekday': days_left,
                         'date': pd.date_range(dt.datetime.today(), periods=7, freq='D').strftime('%Y-%m-%d'),
                         'choice': choice, 'bouts': base_bouts})
    days_extra_bouts = days_left.sample(extra_bouts)
    for i in days_extra_bouts:
        plan.loc[plan['weekday'] == i, 'bouts'] += 1
    return plan

get_a1_plan()








# User's goal and PA choice
goal = 800
choice = "VPA"

# retrieve daily MET of the last week
lastweekMET = SumofMETbyWeekday.loc[SumofMETbyWeekday['week'] == check_even['week'].iloc[-1]]
# print(lastweekMET)

daysPArequired = lastweekMET.loc[lastweekMET.sedentary == 1, 'weekday']
# print(daysPArequired)

METrequired = goal - lastweekMET.MET.sum()
# select x days out of daysPArequired
if choice == 'MVPA':
    numofBouts = math.ceil(METrequired/(4 * 15))
elif choice == 'VPA':
    numofBouts = math.ceil(METrequired/(8 * 15))
    
baseBouts = math.floor(numofBouts/daysPArequired.count())
extraBouts = numofBouts % daysPArequired.count()
plan = pd.DataFrame({'weekday': daysPArequired.tolist(), 'choice': choice, 'bouts': baseBouts})
daysMoreBouts = daysPArequired.sample(extraBouts)
for i in daysMoreBouts:
    plan.loc[plan['weekday'] == i, 'bouts'] += 1
print(plan)

IndentationError: expected an indented block (<ipython-input-198-a35f93230e11>, line 49)

In [ ]:
##############
# < Script 3 #
##############

### Determine the time to send break alert

In [147]:
##############
# Script 4 > #
##############
# input:
# 1. userData
# for now put in testData to test
userSed = copy.deepcopy(userData)

def preprocessData(userData=userSed):
    sedentarySelect = (userData.heart > 0) & (userData.step == 0) & (userData.time_count > 32)
    userData['sedentarySelect'] = sedentarySelect
    userData['sedentaryAccum'] = 0
    try:
        userData = userData.reset_index()
    except ValueError:
        print("index resetted")
    return userData
userSed = preprocessData()
userSed

,index,date,heart,step,time,time_count,user_id,weekday,week,sedentarySelect,sedentaryAccum
0,00:00:00,2016-08-29,89,108,00:00:00,0,a001,0,35,False,0
1,00:15:00,2016-08-29,75,132,00:15:00,1,a001,0,35,False,0
2,00:30:00,2016-08-29,63,0,00:30:00,2,a001,0,35,False,0
3,00:45:00,2016-08-29,65,25,00:45:00,3,a001,0,35,False,0
4,01:00:00,2016-08-29,60,7,01:00:00,4,a001,0,35,False,0
5,01:15:00,2016-08-29,55,0,01:15:00,5,a001,0,35,False,0
6,01:30:00,2016-08-29,57,32,01:30:00,6,a001,0,35,False,0
7,01:45:00,2016-08-29,56,10,01:45:00,7,a001,0,35,False,0
8,02:00:00,2016-08-29,55,0,02:00:00,8,a001,0,35,False,0
9,02:15:00,2016-08-29,59,0,02:15:00,9,a001,0,35,False,0


In [148]:
testData = preprocessData(userData=testData)
testData

,index,date,heart,step,time,time_count,user_id,weekday,week,sedentarySelect,sedentaryAccum
0,00:00:00,2016-09-26,60,0,00:00:00,0,a001,0,39,False,0
1,00:15:00,2016-09-26,60,0,00:15:00,1,a001,0,39,False,0
2,00:30:00,2016-09-26,63,0,00:30:00,2,a001,0,39,False,0
3,00:45:00,2016-09-26,70,114,00:45:00,3,a001,0,39,False,0
4,01:00:00,2016-09-26,53,0,01:00:00,4,a001,0,39,False,0
5,01:15:00,2016-09-26,51,0,01:15:00,5,a001,0,39,False,0
6,01:30:00,2016-09-26,52,0,01:30:00,6,a001,0,39,False,0
7,01:45:00,2016-09-26,55,0,01:45:00,7,a001,0,39,False,0
8,02:00:00,2016-09-26,59,0,02:00:00,8,a001,0,39,False,0
9,02:15:00,2016-09-26,53,0,02:15:00,9,a001,0,39,False,0


In [152]:
# screen sedentary bouts >= 90min
# TODO: change sedentarySelect to sedentary
def CountSedentary(userDT):
    m = 0
    n = 1
    while n < len(userDT):
        if userDT.loc[m, 'sedentarySelect']:
            if userDT.loc[n, 'sedentarySelect']:
                # check if consecutive
                t_1 = pd.to_datetime(userDT.loc[n - 1, 'date'] + ' ' + userDT.loc[n - 1, 'time'])            
                t_2 = pd.to_datetime(userDT.loc[n, 'date'] + ' ' + userDT.loc[n, 'time'])
                timeDelta = t_2 - t_1
                if timeDelta == dt.timedelta(minutes=15):
                    userDT.loc[m, 'sedentaryAccum'] += 1
                    n += 1
                else:
                    m = n
                    n = m + 1
            else:
                m = n + 1
                n = m + 1
        else:
            m = n
            n += 1
            
    return(userDT)


In [153]:
userSed = CountSedentary(userSed)
testData = CountSedentary(testData)

In [154]:
userSed

,index,date,heart,step,time,time_count,user_id,weekday,week,sedentarySelect,sedentaryAccum
0,00:00:00,2016-08-29,89,108,00:00:00,0,a001,0,35,False,0
1,00:15:00,2016-08-29,75,132,00:15:00,1,a001,0,35,False,0
2,00:30:00,2016-08-29,63,0,00:30:00,2,a001,0,35,False,0
3,00:45:00,2016-08-29,65,25,00:45:00,3,a001,0,35,False,0
4,01:00:00,2016-08-29,60,7,01:00:00,4,a001,0,35,False,0
5,01:15:00,2016-08-29,55,0,01:15:00,5,a001,0,35,False,0
6,01:30:00,2016-08-29,57,32,01:30:00,6,a001,0,35,False,0
7,01:45:00,2016-08-29,56,10,01:45:00,7,a001,0,35,False,0
8,02:00:00,2016-08-29,55,0,02:00:00,8,a001,0,35,False,0
9,02:15:00,2016-08-29,59,0,02:15:00,9,a001,0,35,False,0


In [ ]:
# as accumulation starts from 2 consecutive 15min sedentary bouts, 5 means 90min of sedentary time
userData[userData['sedentaryAccum'] >= 5]
userData[userData['date'] == '2016-08-24']

In [155]:
def fillInSedBouts(userData=userSed):
    # fill in sedentary bouts >= 90 min with 1
    userData['sedentaryBouts'] = 0
    i = 0
    while i < len(userData):
        n_bout = userData.sedentaryAccum[i]
        if n_bout >= 5: # >= 90min
            j = i
            while j <= i + n_bout:
                userData.loc[j, 'sedentaryBouts'] = 1
                j += 1
        i += 1
    return userData
            
userSed = fillInSedBouts()
testData = fillInSedBouts(userData=testData)

In [156]:
userSed

,index,date,heart,step,time,time_count,user_id,weekday,week,sedentarySelect,sedentaryAccum,sedentaryBouts
0,00:00:00,2016-08-29,89,108,00:00:00,0,a001,0,35,False,0,0
1,00:15:00,2016-08-29,75,132,00:15:00,1,a001,0,35,False,0,0
2,00:30:00,2016-08-29,63,0,00:30:00,2,a001,0,35,False,0,0
3,00:45:00,2016-08-29,65,25,00:45:00,3,a001,0,35,False,0,0
4,01:00:00,2016-08-29,60,7,01:00:00,4,a001,0,35,False,0,0
5,01:15:00,2016-08-29,55,0,01:15:00,5,a001,0,35,False,0,0
6,01:30:00,2016-08-29,57,32,01:30:00,6,a001,0,35,False,0,0
7,01:45:00,2016-08-29,56,10,01:45:00,7,a001,0,35,False,0,0
8,02:00:00,2016-08-29,55,0,02:00:00,8,a001,0,35,False,0,0
9,02:15:00,2016-08-29,59,0,02:15:00,9,a001,0,35,False,0,0


In [159]:
testData[testData['sedentaryAccum'] > 0]

,index,date,heart,step,time,time_count,user_id,weekday,week,sedentarySelect,sedentaryAccum,sedentaryBouts
42,10:30:00,2016-09-26,73,0,10:30:00,42,a001,0,39,True,2,0
56,14:00:00,2016-09-26,77,0,14:00:00,56,a001,0,39,True,3,0
65,16:15:00,2016-09-26,75,0,16:15:00,65,a001,0,39,True,2,0
129,08:15:00,2016-09-27,59,0,08:15:00,33,a001,1,39,True,1,0
136,10:00:00,2016-09-27,76,0,10:00:00,40,a001,1,39,True,2,0
140,11:00:00,2016-09-27,72,0,11:00:00,44,a001,1,39,True,3,0
148,13:00:00,2016-09-27,82,0,13:00:00,52,a001,1,39,True,2,0
153,14:15:00,2016-09-27,65,0,14:15:00,57,a001,1,39,True,1,0
158,15:30:00,2016-09-27,73,0,15:30:00,62,a001,1,39,True,1,0
162,16:30:00,2016-09-27,59,0,16:30:00,66,a001,1,39,True,1,0


# Method 1: predict the time when sedentary bouts >= 90min start

In [172]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

# feature: time_count, weekday, week?
features = ['time_count', 'weekday', 'week']

y = userSed['sedentaryAccum']
X = userSed[features]
dt1 = DecisionTreeClassifier(min_samples_split=20, random_state=99)
dt1 = dt1.fit(X, y)

In [173]:

dt1.feature_importances_

array([ 0.81013585,  0.07914166,  0.11072249])

In [ ]:
dt1.classes_

In [174]:
X_test = testData[features]
y_test = testData['sedentaryAccum']

## Accuracy of method 1: 

In [175]:
dt1.score(X_test,y_test)

0.9308035714285714

In [185]:
# compare two test data and predicted data manually (conclusion: this is how score is calculated)
predicted_sedentaryAccum = dt1.predict(X_test)
len(predicted_sedentaryAccum)
len(y_test)

correct = 0
for i in range(len(y_test)):
    if predicted_sedentaryAccum[i] == y_test[i]:
        correct += 1
print('accuracy is %f' % (correct/1344))

accuracy is 0.930804


In [184]:
predicted_sedentaryAccum[0]

0

In [176]:
dt1.score(X, y)

0.93787202380952384

In [ ]:
print(dt1.predict([[56, 0, 42]]))
dt1.predict_proba([[62, 0, 42]])

# Plan 2 convert true/false into 0,1; predict 0,1 and find when the user will have prolonged sedentary bouts

In [186]:
import numpy as np
Y_sedentary = np.where(userSed.sedentarySelect == True,1,0)
userSed['sedentary'] = Y_sedentary

In [187]:
features = ['time_count', 'weekday', 'week']
print("* features:", features, sep="\n")

y2 = userSed['sedentary']
X2 = userSed[features]
dt2 = DecisionTreeClassifier(min_samples_split=20, random_state=99)
dt2 = dt2.fit(X2, y2)

* features:
['time_count', 'weekday', 'week']


In [188]:
dt2.feature_importances_

array([ 0.76508659,  0.08747916,  0.14743425])

In [189]:
test_sedentary = np.where(testData.sedentarySelect == True,1,0)
testData['sedentary'] = test_sedentary
X_test = testData[features]
y_test = testData['sedentary']
dt1.score(X_test,y_test)

0.70907738095238093

In [ ]:
dt2.classes_

In [ ]:
print(dt2.predict([[80, 0, 42]]))
dt2.predict_proba([[72, 0, 42]])

### ! pull data everyday
### ! threshold for error message

In [ ]:
dt2.score(X2,y2)

In [ ]:
BreakPlan = pd.DataFrame({'date': np.repeat(pd.date_range(today, periods=7, freq='D').strftime('%Y-%m-%d'), 96, axis=0),
                          'weekday': np.repeat(pd.Series(range(7)).tolist(), 96),
                          'time': pd.date_range(today, periods=96, freq='15min').strftime('%H:%M:%S').tolist() * 7,
                          'time_count': pd.Series(range(96)).tolist() * 7,
                          'week': pd.to_datetime(today).week})
predict_features = list(userData.columns[5:8])
print("* features:", features, sep="\n")

X_predict = BreakPlan[features]
predicted_sedentary = dt2.predict(X_predict)
BreakPlan['sedentary'] = predicted_sedentary
# filter out sleeping time
BreakPlan.loc[BreakPlan['time_count'] < 32, 'sedentary'] = 0
def CountSedentary2(userDT):
    m = 0
    n = 1
    userDT['sedentaryAccum'] = 0
    while n < len(userDT):
        if userDT.loc[m, 'sedentary'] == 1:
            if userDT.loc[n, 'sedentary'] == 1:
                # check if consecutive
                t_1 = pd.to_datetime(userDT.loc[n - 1, 'date'] + ' ' + userDT.loc[n - 1, 'time'])            
                t_2 = pd.to_datetime(userDT.loc[n, 'date'] + ' ' + userDT.loc[n, 'time'])
                timeDelta = t_2 - t_1
                if timeDelta == dt.timedelta(minutes=15):
                    userDT.loc[m, 'sedentaryAccum'] += 1
                    n += 1
                else:
                    m = n
                    n = m + 1
            else:
                m = n + 1
                n = m + 1
        else:
            m = n
            n += 1
            
    return(userDT)

BreakPlan = CountSedentary2(BreakPlan)
BreakPlan['sedentaryBouts'] = 0
i = 0
while i < len(BreakPlan):
    n_bout = BreakPlan.sedentaryAccum[i]
    if n_bout >= 5: # >= 90min
        j = i
        while j <= i + n_bout:
            BreakPlan.loc[j, 'sedentaryBouts'] = 1
            j += 1
    i += 1
    
BreakPlan[BreakPlan['sedentaryBouts'] > 0]

In [ ]:
BreakPlan[(BreakPlan['date'] == '2016-09-30') & (BreakPlan['sedentary'] == 1)]

In [ ]:
BreakPlan[BreakPlan['sedentary'] > 0]

In [ ]:
grouped = BreakPlan.groupby(['week','weekday'])
grouped['sedentary'].sum()
BreakPlan[BreakPlan['weekday'] == 0][28:]

# Plan 3 fill bouts >= 90min with 1 and predict bouts >= 90min

In [192]:
features = ['time_count', 'weekday', 'week']
print("* features:", features, sep="\n")

y3 = userSed['sedentaryBouts']
X3 = userSed[features]
dt3 = DecisionTreeClassifier(min_samples_split=i, random_state=99)
dt3 = dt3.fit(X3, y3)
print('accuracy of prediction: %f' % dt3.score(X_test,testData['sedentaryBouts']))


* features:
['time_count', 'weekday', 'week']
accuracy of prediction: 0.918899


In [ ]:
dt3.feature_importances_

In [ ]:
dt3.classes_

In [ ]:
print(dt3.predict([[56, 0, 42]]))
dt3.predict_proba([[56, 0, 42]])

In [ ]:
range(7)

In [ ]:
# construct dataframe for writing break-sedentary data

BreakPlan = pd.DataFrame({'date': np.repeat(pd.date_range(today, periods=7, freq='D').strftime('%Y-%m-%d'), 96, axis=0),
                          'weekday': np.repeat(pd.Series(range(7)).tolist(), 96),
                          'time': pd.date_range(today, periods=96, freq='15min').strftime('%H:%M:%S').tolist() * 7,
                          'time_count': pd.Series(range(96)).tolist() * 7,
                          'week': pd.to_datetime(today).week})
BreakPlan

In [ ]:
predict_features = list(userData.columns[5:8])
print("* features:", features, sep="\n")

X_predict = BreakPlan[features]
predicted_sedentaryBouts = dt3.predict(X_predict)

In [ ]:
BreakPlan['sedentaryBouts'] = predicted_sedentaryBouts
BreakPlan[BreakPlan['sedentaryBouts'] > 0]

In [ ]:
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")


In [ ]:
visualize_tree(dt, features)

In [ ]:
userData[userData['date'] == '2016-09-01']

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit([SumofMVPAbyWeekday['weekday']], SumofMVPAbyWeekday['MET'])

In [ ]:
import numpy as np
from detect_peaks import detect_peaks

# plot to see

In [ ]:
from bokeh.charts import BoxPlot, Bar, Scatter, output_file, show
from bokeh.io import output_notebook

#### daily total steps

In [ ]:
step_weekday_grouped = userData.groupby(['week', 'weekday'])
SumofstepbyWeekday = step_weekday_grouped['step'].sum()
print(SumofstepbyWeekday.index)

len(SumofstepbyWeekday.index)

In [ ]:
output_notebook()
for i in range(7):
    DayData = userData[userData['weekday'] == i]
    s = Scatter(DayData, y='step', x='time_count',
            title=str(i), plot_width=1000, legend=False)
    show(s)

In [ ]:
Mon = userData[userData['weekday'] == 0]
p = BoxPlot(Mon, values='step', label='time_count',
            title="MVPA of Mon 3weeks", plot_width=1200, legend=False)
b = Bar(userData, values='MVPA', label='weekday', agg='sum',
            title="MVPA of Mon 3weeks", plot_width=1200, legend=False)
s = Scatter(Mon, y='step', x='time_count',
            title="MVPA of Mon 3weeks", plot_width=1200, legend=False)
# output_file("boxplot.html")
output_notebook()
show(p)
show(b)
show(s)





In [ ]:
from bokeh.charts import Bar, output_file, show
from bokeh.sampledata.autompg import autompg as df

p = Bar(userData, label='time_count', values='MVPA', agg='mean',
        title="activity by 15-min", plot_width=1200)

output_file("bar.html")

show(p)

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit([userData['time_count'], userData['activity']], )

In [ ]:
clf.predict([1])